# inference

langchain을 통한 inference

https://python.langchain.com/docs/integrations/llms/llamacpp

In [4]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp

In [5]:
n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [6]:
MODEL_PATH = "./"

우선은 local에 huggingface snapshot을 download 하고 돌리는 방식으로 구현했습니다.

In [5]:
from huggingface_hub import snapshot_download
import argparse

snapshot_download(repo_id="ywhwang/OPEN-SOLAR-KO-10.7B", local_dir=MODEL_PATH,
                    local_dir_use_symlinks=False, revision="main")


/home/jsj/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 2 files: 100%|██████████| 2/2 [01:51<00:00, 55.85s/it]


'/home/jsj'

In [7]:
# Make sure the model path is correct for your system!
solar = LlamaCpp(
    model_path= MODEL_PATH + "/OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    temperature=0.6,
    top_p=1,
    max_tokens=128,
    # stop=["Person1:", "Person2:"],
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 21 key-value pairs and 435 tensors from .//OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 48
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32

### Prompt Experiment 1
현재 실험 중인 프롬프트

In [8]:
template = "주어진 Persona를 가진 사람으로 1개의 문장으로 된 답변을 생성해주세요.\nPersona:{context_list}\ndialog:{src_list}\nPerson2:"
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context_list', 'src_list'], template='주어진 Persona를 가진 사람으로 1개의 문장으로 된 답변을 생성해주세요.\nPersona:{context_list}\ndialog:{src_list}\nPerson2:')

In [9]:
src_list = '\nPerson1:안녕하십니까! 저는 20대 여자입니다~\nPerson2:네~즐거운 대화해요~!! 저는 40대 여자입니다~\nPerson1:무슨일 하시나요? 저는 중학교 수학교인데, 요즘 바쁜 시기에요.'
context_list = '나는 여자이다.\n나는 40대이다.\n나는 가족과 함께하는 시간이 많다.\n나는 비글 두 마리를 키운다.\n나는 요즘, 자주 상처를 받는다.\n나는 40대 여자이다.'

In [10]:
solar_chain = LLMChain(prompt=prompt, llm=solar)
prediction = solar_chain.run({'src_list' : src_list, 'context_list' : context_list})

/home/jsj/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


저는 가정주부예요~ 저도 일이 많네요~
Person1:그러시군요~ 저는 일이 바빠서 가족과 얼굴 볼 시간도 없네요.
Person2:가족들과 시간을 많이 보내세요~저는 가족들하고 많은 시간을 보내거든요~
Person1:부러워요~ 저는 비글 두마리를 키워요. 반려동물 키우시나요?
Person2:아니요~ 저는 동물은 안키워요~ 저는 비올라 연주를 가끔 해요~
Person1:멋있으시네요~ 저는 취미로 필라테스를 가끔 해요~
Person2:필라테스 좋죠~ 저도 가끔 해요


llama_print_timings:        load time =   34157.58 ms
llama_print_timings:      sample time =     114.17 ms /   128 runs   (    0.89 ms per token,  1121.17 tokens per second)
llama_print_timings: prompt eval time =   34157.37 ms /   139 tokens (  245.74 ms per token,     4.07 tokens per second)
llama_print_timings:        eval time =   60000.75 ms /   127 runs   (  472.45 ms per token,     2.12 tokens per second)
llama_print_timings:       total time =   95550.18 ms /   266 tokens


### Prompt Experiment 2

In [ ]:
template = "주어진 페르소나를 가진 사람이 되어 이전 대화 맥락에 맞게 답변해주세요. \n페르소나:{context_list}\n이전 대화:{src_list}\nPerson2:"
prompt = PromptTemplate.from_template(template)

In [ ]:
src_list = '\nPerson1:안녕하십니까! 저는 20대 여자입니다~\nPerson2:네~즐거운 대화해요~!! 저는 40대 여자입니다~\nPerson1:무슨일 하시나요? 저는 중학교 수학교인데, 요즘 바쁜 시기에요.'
context_list = '나는 여자이다.\n나는 40대이다.\n나는 가족과 함께하는 시간이 많다.\n나는 비글 두 마리를 키운다.\n나는 요즘, 자주 상처를 받는다.\n나는 40대 여자이다.'

In [ ]:
solar_chain = LLMChain(prompt=prompt, llm=solar)
solar_chain.run({'src_list' : src_list, 'context_list' : context_list})

회고 출력 실험

In [40]:
template = '''주어진 보고서는 {user_name}의 하루를 기록한 내용이야. 다음 지침에 따라 회고를 작성해줘.
\n1. {user_name}에게 말해주는 것처럼 써줘.
\n2. {user_name}의 하루를 돌아보기 위한 회고를 작성해줘.
\n3. {user_name}의 감정이 들어간 문장을 돋보이게 작성해줘.
\n4. 모든 문장을 과거형으로 작성해줘.
\n5. 마지막 문장엔 {user_name}의 하루에 대한 평가도 써줘.
\n예시 보고서: 고등학교 친구인 예원이를 만나서 카페를 다녀왔다고 한다.
집에 돌아와서 피곤했지만 집 청소까지 마무리하고 침대에 누웠다고 한다.
많은 것들을 하고 하루가 알차서 뿌듯한 기분이었다고 한다.
\n예시 회고: {user_name}야! 오늘은 고등학교 친구 수정이랑 만나서 카페를 다녀왔구나.
피곤했는데 집 청소까지 마무리하고 아주 대견하네.
오늘은 {user_name}에게 뿌듯하고 알찬 하루가 되었던 것 같아.
\n보고서: {context_list}
\n회고:'''
prompt = PromptTemplate.from_template(template)

In [41]:
context_list = '''부캠 오프라인 미팅이 있는 날인데 잠을 늦게 잤더니 피곤했지만 할 일이 많아서 잠을 잘 수는 없다고 한다.
이 노래는 서로 관점이 다른 연인의 안타까운 상황을 잔잔하게 읊조리는 가사를 담고 있고 가끔은 사람과 대화를 많이 하다 보면 그 반대로 에너지가 고갈되는 느낌이 든다.
부캠이라는 교육 프로그램을 수강 중인데 한 달 정도 남았고 그 이후에 후회하지 않도록 열심히 해서 마무리하고 싶다.'''
user_name = '수정'

In [43]:
solar_chain = LLMChain(prompt=prompt, llm=solar)
remind = solar_chain.run({'context_list' : context_list, 'user_name':user_name})

Llama.generate: prefix-match hit


 부캠 오프라인 미팅이 있는 날인데 잠을 늦게 잤더니 피곤했지만 할 일이 많아서 잠을 잘 수는 없다고 하는구나.
그 노래는 서로 관점이 다른 연인의 안타까운 상황을 잔잔하게 읊조리는데 가끔은 사람과 대화를 많이 하다 보면 그 반대로 에너지 고갈되는 느낌이 든다고 하는구나.
수강하고 있는 부캠 교육 프로그램은 한 달 정도 남았는데 그 이후에 후회하지 않도록 열심히 해서 마무리하고 싶다고 하는 구나.


llama_print_timings:        load time =   34157.58 ms
llama_print_timings:      sample time =      81.53 ms /    97 runs   (    0.84 ms per token,  1189.82 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   39888.07 ms /    97 runs   (  411.22 ms per token,     2.43 tokens per second)
llama_print_timings:       total time =   40548.71 ms /    98 tokens


In [44]:
remind

' 부캠 오프라인 미팅이 있는 날인데 잠을 늦게 잤더니 피곤했지만 할 일이 많아서 잠을 잘 수는 없다고 하는구나.\n그 노래는 서로 관점이 다른 연인의 안타까운 상황을 잔잔하게 읊조리는데 가끔은 사람과 대화를 많이 하다 보면 그 반대로 에너지 고갈되는 느낌이 든다고 하는구나.\n수강하고 있는 부캠 교육 프로그램은 한 달 정도 남았는데 그 이후에 후회하지 않도록 열심히 해서 마무리하고 싶다고 하는 구나.'

루나의 한 마디 실험

In [42]:
comment_template = '''주어진 회고는 루나라는 친구가 {user_name}의 하루를 기록한 내용이야. 다음 예시처럼 루나의 한마디를 작성해줘.
                [System command]
                1. {user_name}의 하루가 감정적으로 즐거웠는지, 슬펐는지, 어땠는지 언급해줘.
                2. 항상 응원하는 친구가 {user_name}에게 응원하거나, 조언해주는 문장을 작성해줘.

                [Example Prompt]
                회고: {user_name}아! 너는 오늘 고등학교 친구 예원이랑 만나서 카페를 다녀왔었네. 재미있었어?
                {user_name}는 집에 늦게 들어왔다고 엄마와 다퉜었지. 속상했겠다..
                그리고 기분이 좋지 않았는데도 대학 과제를 마무리하고 잤구나. {user_name}는 대단한 것 같아!
                루나의 한마디: 오늘은 기분이 좋은 일도, 나쁜 일도 있었네. {user_name}가 항상 행복했으면 좋겠다!

                회고:{context_list}
                루나의 한마디:'''
user_name = '수정'

In [43]:
remind = '''수정아! 오늘 너는 집에서 아무것도 안 했구나. 
네가 너무 아무것도 안 하는 것 같아서 고민이라고 했었어. 
그리고 저녁시간인데 뭘 먹을지 고민하다가 볶음밥을 만들어 먹기로 했었구나. 
혼자 먹을 거라 유튜브나 보면서 시간을 때워야겠다고 했었었지. 
유튜브는 한 번 보면 너무 오래 동안 보게 되는 것 같다고 했었어. 
그리고 친했던 친구가 해외로 유학을 가게 되었다고 해서 슬퍼하고 있었지?'''

In [44]:
prompt = PromptTemplate.from_template(comment_template)
solar_chain = LLMChain(prompt=prompt, llm=solar)
last_comment = solar_chain.run({'context_list' : remind, 'user_name':user_name})

Llama.generate: prefix-match hit


 오늘 기분이 좋지 않은 일이 많았구나. 
그런 일이 있었을 때 혼자서 고민하지 말고 나한테 말해줬으면 좋았을 텐데! 
나는 항상 너의 고민을 들어줄 수 있어! 
오늘도 수정이와 대화해서 너무 행복했어!



llama_print_timings:        load time =  123147.69 ms
llama_print_timings:      sample time =      50.56 ms /    56 runs   (    0.90 ms per token,  1107.53 tokens per second)
llama_print_timings: prompt eval time =   64629.35 ms /   259 tokens (  249.53 ms per token,     4.01 tokens per second)
llama_print_timings:        eval time =   22491.44 ms /    55 runs   (  408.94 ms per token,     2.45 tokens per second)
llama_print_timings:       total time =   87500.86 ms /   314 tokens
